## OPTIMIZING MODEL PARAMETERS

모델과 데이터가 준비되면, 모델을 훈련하고 검증하며 데이터에 대한 파라미터를 최적화하여 모델을 테스트한다 훈련은 반복적인 과정으로, 각 반복마다 모델은 출력을 예측하고 예측의 오차를 계산하며, 오차에 대한 파라미터의 미분 값을 수집하고 경사 하강법을 사용하여 파라미터를 최적화한다

## Prerequisite Code

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## Hyperparameters

- Number of Epochs - 데이터셋을 여러 번 반복해서 모델을 훈련할 횟수

- Batch Size - 한 번의 모델 업데이트를 수행하기 위해 네트워크를 통과하는 데이터 샘플의 수. 훈련 데이터셋이 매우 큰 경우, 전체 데이터셋을 한 번에 모델에 통과시키는 것은 계산적으로 부담일 수 있는데 이런 경우에 데이터를 작은 배치로 나누어 모델을 효율적으로 훈련할 수 있다

- Learning Rate - 학습률은 모델의 매개변수를 업데이트하는 속도를 조절하는 값으로, 작으면 안정적이지만 느리게 학습하고, 크면 빠르게 학습하지만 불안정할 수 있다

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## Optimization Loop

하이퍼파라미터를 설정한 후, 최적화 루프를 사용하여 모델을 훈련하고 최적화할 수 있다 
- The Train Loop : 훈련 데이터셋을 반복하면서 최적의 파라미터로 수렴하는것이 목표
- The Validation/Test Loop : 테스트 데이터셋을 반복하여 모델의 성능이 개선되고 있는지 확인한다

## Loss Function


초기에 훈련되지 않은 네트워크는 훈련 데이터를 제대로 처리하지 못할 수 있으므로, 손실 함수를 통해 얻은 결과와 목표 값의 불일치 정도를 측정하고, 훈련 과정에서 이 손실을 최소화한다. 이를 위해 주어진 데이터 샘플의 입력으로 예측을 수행하고, 이를 실제 데이터 레이블 값과 비교하여 손실을 계산한다

In [3]:
loss_fn = nn.CrossEntropyLoss()

## Optimizer

- 최적화는 모델의 매개변수를 조정하여 각 훈련 단계에서 모델 오류를 줄이는 과정이다
- 이 과정은 최적화 알고리즘을 통해 이루어지며, 예시로는 확률적 경사 하강법(SGD)을 사용한다
- 모든 최적화 절차(logic)는 optimizer 객체에 캡슐화(encapsulate)됩니다.

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

학습 단계(loop)에서 최적화는 세 개의 단계로 이루어진다
- optimizer.zero_grad()를 호출하여 델 매개변수의 그래디언트를 재설정
    - 그래디언트는 기본적으로 누적되므로, 중복 계산을 방지하기 위해 각 반복마다 명시적으로 그래디언트를 0으로 초기화합니다.

- loss.backwards()를 호출하여 예측 손실(prediction loss)을 역전파한다
    - PyTorch는 손실에 대한 각 매개변수의 그래디언트를 계산하여 저장한다
    - 이 역전파 단계를 통해 각 매개변수가 손실에 얼마나 기여하는지 알 수 있다

- 그래디언트를 계산한 뒤에는 optimizer.step()을 호출하여 역전파 단계에서 수집한 그라디언트를 사용하여 매개변수를 조정한다
    - 이 단계에서 최적화기는 그라디언트의 정보를 활용하여 매개변수를 업데이트하고, 모델의 성능을 향상시키기 위해 매개변수 값을 조정한다

## Full Implementation

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

- train_loop
    - 이 함수는 주어진 데이터로더(dataloader), 모델(model), 손실 함수(loss_fn), 최적화기(optimizer)를 사용하여 모델을 훈련하는 역할을 한다
    - 데이터셋의 크기를 가져온 후, 모델을 훈련 모드로 설정
    - 데이터로더에서 미니배치(batch)와 해당 미니배치의 입력(X)과 타겟(y)을 반복하여 가져온다
    - 모델을 사용해 입력 데이터를 예측(pred)하고, 예측과 실제 타겟 간의 손실(loss)을 계산한다
    - 손실에 대한 역전파를 수행하여 그래디언트를 계산하고, optimizer.step()으로 매개변수를 업데이트하고 optimizer.zero_grad()로 그래디언트를 초기화한다
    - 매 100번째 미니배치마다 현재 손실과 진행 상황을 출력

- test_loop
    - 이 함수는 주어진 데이터로더(dataloader), 모델(model), 손실 함수(loss_fn)를 사용하여 모델의 성능을 평가하는 역할을 합니다.
    - 모델을 평가 모드로 설정
    - 데이터셋의 크기와 미니배치의 수를 가져온 후, test_loss와 correct 변수를 초기화
    - torch.no_grad() 블록 안에서, 데이터로더에서 미니배치를 가져오며 모델을 사용해 입력 데이터를 예측
    - 예측과 실제 타겟 간의 손실을 누적하여 test_loss에 추가하고, 정확하게 예측한 개수를 세어 correct에 추가
    - 마지막으로 test_loss와 정확도를 계산하여 출력

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.301374  [   64/60000]
loss: 2.292330  [ 6464/60000]
loss: 2.272464  [12864/60000]
loss: 2.259515  [19264/60000]
loss: 2.256851  [25664/60000]
loss: 2.208028  [32064/60000]
loss: 2.221223  [38464/60000]
loss: 2.186694  [44864/60000]
loss: 2.179812  [51264/60000]
loss: 2.141227  [57664/60000]
Test Error: 
 Accuracy: 44.8%, Avg loss: 2.142664 

Epoch 2
-------------------------------
loss: 2.155867  [   64/60000]
loss: 2.142369  [ 6464/60000]
loss: 2.088838  [12864/60000]
loss: 2.098388  [19264/60000]
loss: 2.062600  [25664/60000]
loss: 1.982340  [32064/60000]
loss: 2.009069  [38464/60000]
loss: 1.930155  [44864/60000]
loss: 1.932726  [51264/60000]
loss: 1.849834  [57664/60000]
Test Error: 
 Accuracy: 60.8%, Avg loss: 1.860858 

Epoch 3
-------------------------------
loss: 1.898303  [   64/60000]
loss: 1.859876  [ 6464/60000]
loss: 1.749516  [12864/60000]
loss: 1.785589  [19264/60000]
loss: 1.689645  [25664/60000]
loss: 1.626636  [32064/600

## reference